In [ ]:
%matplotlib inline
from lixtools.hdf import h5xs_scan
import pylab as plt
import numpy as np
from dask.distributed import Client
from IPython.display import display,HTML
import warnings

warnings.filterwarnings('ignore')

fn = "00template00.h5"
dt = h5xs_scan(fn)
dt.load_data()

client = Client(n_workers=8)

# show averaged data, pick up to 3 raw data files
samples = list(dt.h5xs.keys())
N = 1
if len(samples)>4:
    N = 2
if len(samples)>7:
    N = 3
for i in np.linspace(0,len(samples)-1,N, dtype=int):
    sn = samples[i]
    print(f"plotting {sn} ...")
    #display(HTML(sn))
    fig = plt.figure(figsize=(11,5))
    dt.h5xs[sn].show_data(frn="average", fig=fig, showRef=None, cmap='jet', client=client)
    plt.show()

if len(samples)>1:
    sn = "overall"
else:
    sn = samples[0]
# show the maps, two or three per row
asp = {"maps": "auto", "tomo": 1}
for k in ["maps", "tomo"]:
    print("\n\n", k)
    sks = list(dt.proc_data[sn][k].keys())
    if "transmission" in sks:
        sks.remove("transmission")   # go with absoprtion instead
    nsk = len(sks)
    if nsk>4:
        nc = 3
    else:
        nc = 2
    nr = int(np.ceil(nsk/nc))
    fig = plt.figure(figsize=(nc*5, nr*5))
    for i in range(nsk):
        ax = fig.add_subplot(nr, nc, i+1)
        dt.proc_data[sn][k][sks[i]].plot(ax=ax, aspect=asp[k], cmap='jet')
        ax.set_title(sks[i])
    plt.subplots_adjust(wspace=0.375, hspace=0.375)
    plt.show()